In [1]:
from osgeo import ogr
import shapely, fiona, json

In [2]:
raw_strings = sc.textFile("all_tweets_one_file.jsonl") # Would Using HDFS be smarter here?

In [3]:
raw_strings.take(1)

[u'{"geometry": {"type": "Point", "coordinates": [-74.5342206, 39.3954102]}, "type": "Feature", "properties": {"u_utc": null, "text": "- I do my own thing , you made me this way .", "source": {"link": "http://twitter.com/download/android", "displayName": "Twitter for Android"}, "verb": "post", "user": "Dougieee_Nastyy", "time": "2012-10-25T11:25:26.000Z", "gnip": {"matching_rules": [{"tag": "[-74.53321912339669,39.36436374931048]", "value": "bounding_box:[-74.57813488760267 39.329638110038545 -74.48830335919072 39.39908938858241]"}], "language": {"value": "en"}}}}']

In [7]:
tweets = raw_strings.map(json.loads)
tweets.cache()

PythonRDD[4] at RDD at PythonRDD.scala:53

In [8]:
t1 = tweets.take(1)[0]

In [9]:
print(t1)

{u'geometry': {u'type': u'Point', u'coordinates': [-74.5342206, 39.3954102]}, u'type': u'Feature', u'properties': {u'u_utc': None, u'text': u'- I do my own thing , you made me this way .', u'source': {u'displayName': u'Twitter for Android', u'link': u'http://twitter.com/download/android'}, u'verb': u'post', u'user': u'Dougieee_Nastyy', u'time': u'2012-10-25T11:25:26.000Z', u'gnip': {u'matching_rules': [{u'tag': u'[-74.53321912339669,39.36436374931048]', u'value': u'bounding_box:[-74.57813488760267 39.329638110038545 -74.48830335919072 39.39908938858241]'}], u'language': {u'value': u'en'}}}}


## Load Zone A as a Geometry

In [32]:
with open('ZoneA_SingleGeometry.geojson','r') as zoneAFile:
    zoneAJSON = json.loads(zoneAFile.read())
    geom = zoneAJSON['features'][0]['geometry']
    zoneA = ogr.CreateGeometryFromJson(json.dumps(geom))

In [53]:
# from shapely.wkt import dumps, loads
# dumps(loads('POINT (0 0)'))
# 'POINT (0.0000000000000000 0.0000000000000000)'
# Shapely can also integrate with other Python GIS packages using GeoJSON-like dicts.


s.is_valid

False

In [89]:
from shapely.geometry import mapping, shape
c = fiona.open('ZoneA/OGRGeoJSON.shp','r')
pol = c.next()
geom = shape(pol['geometry']).buffer(0)
geom.is_valid

True

In [90]:
zoneABroadcast = sc.broadcast(geom)

In [91]:
#Testing:
in_bounds = shape({'type': "Point", 'coordinates': [-73.94900, 40.73389]})
print in_bounds
print( geom.contains(in_bounds))
# print("In Bounds (Should be True): " + str( point_in_bounds.Within(zoneABroadcast.value)))

out_of_bounds = shape({"type": "Point","coordinates": [-73.99753114562988,40.73093368341445]})
print out_of_bounds
print geom.contains(out_of_bounds)

POINT (-73.949 40.73389)
True
POINT (-73.99753114562988 40.73093368341445)
False


In [92]:
def get_point(tweet):
    return shape(tweet["geometry"])

#Test it:
p = get_point(t1)
print(p)
p

POINT (-74.5342206 39.3954102)


### Use Spark to find ZoneA tweets

In [96]:
inZoneA = tweets.filter(lambda t: zoneABroadcast.value.contains(get_point(t)))

In [97]:
# Kick off the Action, save the tweets
inZoneA.cache()
inZoneA.count()

110111

In [98]:
inZoneA.saveAsTextFile('inZoneA.txt')

In [101]:
grouped_by_user = inZoneA.groupBy(lambda t: t['properties']['user'])
count_by_user   = grouped_by_user.map(lambda tu: (tu[0],len(tu[1]) ))
sorted_by_quant = count_by_user.sortBy(lambda tu: -tu[1])
sorted_by_quant.cache()
print("Users: " + str(sorted_by_quant.count()))

Users: 22042


In [ ]:
# Write out all the tweets to GeoJSON

# Now build a secondary GeoJSON server to only serve specific dates?

# for tu in sorted_by_quant.collect():
#     u_tweets = tweets.filter(lambda t: t['properties']['user'] == tu[0])
#     features = u_tweets.collect()
#     with open('zone_a_twitterers/'+tu[0]+'.geojson','w') as outGeo:
#         json.dump({"type":"FeatureCollection","features":features},outGeo)